In [1]:
import ee
import folium
import geehydro
import geemap

C:\Users\srini\AppData\Local\Programs\Python\Python310\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [2]:
ee.Initialize(project='ml-meets-the-sun')

In [3]:
Map = geemap.Map(center = (15.3173,75.7139), zoom =5)
Map.add_basemap('HYBRID')

In [4]:
Map

Map(center=[15.3173, 75.7139], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topri…

In [5]:
karnataka = ee.FeatureCollection('projects/ml-meets-the-sun/assets/Karnataka')
TamilNadu = ee.FeatureCollection('projects/ml-meets-the-sun/assets/TamilNadu')
Rajasthan = ee.FeatureCollection('projects/ml-meets-the-sun/assets/Rajasthan')

In [6]:
def map_display(state,name):
    vis_params = {
    'color': 'red'
    }
    Map.addLayer(state, vis_params, name, True, 0.5)
    return Map
Map

Map(center=[15.3173, 75.7139], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topri…

In [7]:
states = {
    'Karnataka': karnataka,
    'Tamil Nadu': TamilNadu,
    'Rajasthan': Rajasthan
}
for name,state in states.items():
    map_display(state,name)

In [14]:
era5_2mt_rad = (
    ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
    .select('surface_net_solar_radiation')
    .filter(ee.Filter.date('2020-01-01', '2025-07-31'))
)
display(era5_2mt_rad)

In [15]:
era5_2mt_rad.first()

In [16]:
clipped= era5_2mt_rad.first()
vis_temp ={
    'min': 250,
    'max': 320,
    'palette': [
        '000080',
        '0000d9',
        '4000ff',
        '8000ff',
        '0080ff',
        '00ffff',
        '00ff80',
        '80ff00',
        'daff00',
        'ffff00',
        'fff500',
        'ffda00',
        'ffb000',
        'ffa400',
        'ff4f00',
        'ff2500',
        'ff0a00',
        'ff00ff',
    ],
    }
Map.addLayer(clipped, vis_temp, 'cname2', True, 0.5)

In [17]:
era5_rad_mean= era5_2mt_rad.mean().setDefaultProjection(
    era5_2mt_rad.first().projection()
)


In [18]:
result_clip = {}

def mean_rad_clip(state, name):
    cname = name + "_rad_mean"
    clipped = era5_rad_mean.clip(state)
    result_clip[cname] = clipped

In [19]:
for name,state in states.items():
    mean_rad_clip(state, name)

In [20]:
result_clip

{'Karnataka_rad_mean': <ee.image.Image at 0x1bfdbd5ca30>,
 'Tamil Nadu_rad_mean': <ee.image.Image at 0x1bfdbc65540>,
 'Rajasthan_rad_mean': <ee.image.Image at 0x1bfdbc65360>}

In [21]:
def display_mean_rad(cname, clipped):
    
    vis_rad ={
    'min': 250,
    'max': 320,
    'palette': [
        '000080',
        '0000d9',
        '4000ff',
        '8000ff',
        '0080ff',
        '00ffff',
        '00ff80',
        '80ff00',
        'daff00',
        'ffff00',
        'fff500',
        'ffda00',
        'ffb000',
        'ffa400',
        'ff4f00',
        'ff2500',
        'ff0a00',
        'ff00ff',
    ],
    }
    Map.addLayer(clipped, vis_rad, cname, True, 0.5)
    return Map

In [22]:
for cname,clipped in result_clip.items():
    display_mean_rad(cname, clipped)

In [23]:
result_clip['Karnataka_rad_mean'].projection()

In [24]:
region = karnataka.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Karnataka_rad_mean'],
    description='Karnataka_rad_mean',
    assetId='projects/ml-meets-the-sun/assets/KA_mean_rad',
    region=region,
    crsTransform = [0.1, 0, -180.05, 0, -0.1, 90.05],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()

In [25]:
region = TamilNadu.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Tamil Nadu_rad_mean'],
    description='TamilNadu_mean_rad',
    assetId='projects/ml-meets-the-sun/assets/TN_mean_rad',
    region=region,
    crsTransform = [0.1, 0, -180.05, 0, -0.1, 90.05],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()

In [26]:
region = Rajasthan.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Rajasthan_rad_mean'],
    description='Rajasthan_rad_temp',
    assetId='projects/ml-meets-the-sun/assets/RA_mean_rad',
    region=region,
    crsTransform = [0.1, 0, -180.05, 0, -0.1, 90.05],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()


In [27]:
karnataka_temp = ee.Image('projects/ml-meets-the-sun/assets/KA_mean_rad')
TamilNadu_temp = ee.Image('projects/ml-meets-the-sun/assets/TN_mean_rad')
Rajasthan_temp = ee.Image('projects/ml-meets-the-sun/assets/RA_mean_rad')

In [28]:
vis_temp = {
    'min': 250,
    'max': 320,
    'palette': [
        '000080',
        '0000d9',
        '4000ff',
        '8000ff',
        '0080ff',
        '00ffff',
        '00ff80',
        '80ff00',
        'daff00',
        'ffff00',
        'fff500',
        'ffda00',
        'ffb000',
        'ffa400',
        'ff4f00',
        'ff2500',
        'ff0a00',
        'ff00ff',
    ],
}

In [31]:
Map.addLayer(karnataka_temp, vis_temp, 'ERA5_KA_Mean_rad', True, 0.5)
Map.addLayer(TamilNadu_temp, vis_temp, 'ERA5_TN_Mean_rad', True, 0.5)
Map.addLayer(Rajasthan_temp, vis_temp, 'ERA5_RA_Mean_rad', True, 0.5)

In [32]:
Map

Map(center=[15.3173, 75.7139], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topri…